In [ ]:
from itertools import product
import time
import numpy as np
import pandas as pd
from nba_api.stats.endpoints import leaguedashplayerptshot, playerdashptshots, leaguedashteamptshot
from tenacity import RetryError, Retrying, stop_after_attempt, wait_fixed
from tqdm import tqdm

pd.options.mode.chained_assignment = None

data_DIR = "../../data/shots/"

# Each Dash Type Separately

In [ ]:
league, league_id = "NBA", "00"
season_start = 2023
season_end = 2024
seasons = np.arange(season_start, season_end, 1).astype(str)
dash_types = ["overall","shot_type","shot_clock","dribble","closest_def","closest_def_10","touch_time"]
for season in tqdm(seasons):
    season_str = season + "-" + str(int(season)+1)[-2:]
    stats = playerdashptshots.PlayerDashPtShots(league_id = league_id,team_id = 0,player_id = 0,season=season_str).get_data_frames()
    for i,d in enumerate(dash_types):
        df = stats[i].drop(columns=["SORT_ORDER","FGA_FREQUENCY","FG2A_FREQUENCY","FG3A_FREQUENCY"])
        df.to_parquet(data_DIR + f"{league}_Shots_{season}_{d}.parquet")

# Bin Shots by Condition

In [ ]:
general_range = ['Catch and Shoot', 'Pullups', 'Less Than 10 ft']
shot_clock = ['24-22', '22-18 Very Early', '18-15 Early', '15-7 Average', '7-4 Late', '4-0 Very Late']
dribbles = ['0 Dribbles', '1 Dribble', '2 Dribbles', '3-6 Dribbles', '7+ Dribbles']
closest_def = ['0-2 Feet - Very Tight', '2-4 Feet - Tight', '4-6 Feet - Open', '6+ Feet - Wide Open']
touch_time = ['Touch < 2 Seconds', 'Touch 2-6 Seconds', 'Touch 6+ Seconds']

In [ ]:
league, league_id = "NBA", "00"
season_start = 2024
season_end = 2025
seasons = np.arange(season_start, season_end, 1).astype(str)

In [ ]:
for season in seasons:
    season_str = season + "-" + str(int(season)+1)[-2:]
    n = 0
    for a, b, c in product(general_range,closest_def, touch_time):
        n+=1
    dfa = []
    for a, b, c in tqdm(product(general_range,closest_def, touch_time),total=n):
        for i in Retrying(stop=stop_after_attempt(5), wait=wait_fixed(0.6)):
            try:
                stats = leaguedashplayerptshot.LeagueDashPlayerPtShot(
                    league_id = league_id,
                    season=season_str,
                    season_type_all_star = "Regular Season",
                    general_range_nullable = a,
                    close_def_dist_range_nullable= b,
                    touch_time_range_nullable = c, 
                    # dribble_range_nullable =  d,
                ).get_data_frames()
                df1 = stats[0]
                df1["general_range"] = a
                df1["closest_def"] = b
                df1["touch_time"] = c
                dfa.append(df1)
                break
            except Exception as error:
                print(error)
                continue
    dfa1 = [df2 for df2 in dfa if not df2.empty]
    df = pd.concat(dfa1)
    # df.to_parquet(data_DIR + f"{league}_Shots_{season}_All.parquet")

In [ ]:
dfa1 = [df2 for df2 in dfa if not df2.empty]

In [ ]:
len(dfa1)

In [ ]:
for i in range(len(dfa1)):
    if len(dfa1[i])<5:
        display(dfa1[i])

In [ ]:
dfa1 = [df2.fillna(0) for df2 in dfa if not df2.empty]

In [ ]:
df2["FG3_PCT"] = 0

In [ ]:
dfa1 = [df2.infer_objects(copy=False).fillna(0) for df2 in dfa if not df2.empty]
df = pd.concat(dfa1)

In [ ]:
df = pd.concat(dfa1)


## Team

In [ ]:
for season in seasons:
    season_str = season + "-" + str(int(season)+1)[-2:]
    n = 0
    for a, b, c in product(general_range,closest_def, touch_time):
        n+=1
    dfa = []
    for a, b, c in tqdm(product(general_range,closest_def, touch_time),total=n):
        for i in Retrying(stop=stop_after_attempt(5), wait=wait_fixed(0.6)):
            try:
                stats = leaguedashteamptshot.LeagueDashTeamPtShot(
                    league_id = league_id,
                    season=season_str,
                    season_type_all_star = "Regular Season",
                    general_range_nullable = a,
                    close_def_dist_range_nullable= b,
                    touch_time_range_nullable = c, 
                    # dribble_range_nullable =  d,
                ).get_data_frames()
                df1 = stats[0]
                df1["general_range"] = a
                df1["closest_def"] = b
                df1["touch_time"] = c
                dfa.append(df1)
                break
            except Exception as error:
                print(error)
                continue
    dfa1 = [df2 for df2 in dfa if not df2.empty]
    df = pd.concat(dfa1)
    df.to_parquet(data_DIR + f"{league}_Shots_T_{season}_All.parquet")

In [ ]:
df.to_parquet(data_DIR + f"{league}_Shots_T_{season}_All.parquet")

# By Date

In [ ]:
season = "2023"
league, league_id = "NBA", "00"
general_range0 = ['Catch and Shoot']
general_range = ['Pullups', 'Less Than 10 ft']
closest_def = [
    "0-2 Feet - Very Tight",
    "2-4 Feet - Tight",
    "4-6 Feet - Open",
    "6+ Feet - Wide Open",
]
touch_time0 = ["Touch < 2 Seconds"]
touch_time = ["Touch < 2 Seconds","Touch 2-6 Seconds", "Touch 6+ Seconds"]

season_str = season + "-" + str(int(season) + 1)[-2:]
n = 0
for a, b, c in product(general_range, closest_def, touch_time):
    n += 1
dfa = []
from datetime import date, timedelta

d1 = date(2023,10,24)
d2 = date.today()

# this will give you a list containing all of the dates
dd = [d1 + timedelta(days=x) for x in range((d2-d1).days + 1)]
for d in tqdm(dd):
    for a, b, c in product(general_range0, closest_def, touch_time0):
        for i in range(5):
            try:
                stats = leaguedashplayerptshot.LeagueDashPlayerPtShot(
                    league_id=league_id,
                    season=season_str,
                    general_range_nullable=a,
                    close_def_dist_range_nullable=b,
                    touch_time_range_nullable=c,
                    date_from_nullable = d.strftime("%Y-%m-%d"),
                    date_to_nullable = d.strftime("%Y-%m-%d"),

                ).get_data_frames()
                df1 = stats[0]
                df1["general_range"] = a
                df1["closest_def"] = b
                df1["touch_time"] = c
                df1["Date"]=d.strftime("%Y-%m-%d")
                dfa.append(df1)
                time.sleep(0.6)
                break
            except Exception as error:
                print(error)
                # continue
    for a, b, c in product(general_range, closest_def, touch_time):
        for i in range(5):
            try:
                stats = leaguedashplayerptshot.LeagueDashPlayerPtShot(
                    league_id=league_id,
                    season=season_str,
                    general_range_nullable=a,
                    close_def_dist_range_nullable=b,
                    touch_time_range_nullable=c,
                    date_from_nullable = d.strftime("%Y-%m-%d"),
                    date_to_nullable = d.strftime("%Y-%m-%d"),

                ).get_data_frames()
                df1 = stats[0]
                df1["general_range"] = a
                df1["closest_def"] = b
                df1["touch_time"] = c
                df1["Date"]=d.strftime("%Y-%m-%d")
                dfa.append(df1)
                time.sleep(0.6)
                break
            except Exception as error:
                print(error)
                # continue
dfa1 = [df2.fillna(0) for df2 in dfa if not df2.empty]
df = pd.concat(dfa1)

In [ ]:
dfa1 = [df2.fillna(0) for df2 in dfa if not df2.empty]
df = pd.concat(dfa1)

In [ ]:
df[df["Date"] == "2023-10-24"]["general_range"].unique()

In [ ]:
df.to_parquet(data_DIR + f"{league}_Shots_{season}_All_Date.parquet")

In [ ]:
df